In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import neuron
import plotting

import sys

from pathlib import Path
import os

%matplotlib notebook

In [ ]:
sys.path.append('../ecode/')

In [ ]:
from generate_ecode_rheobase import generate_ecode_protocols, compute_rheobase_for_model

In [ ]:
#!rm -r x86_64
#!nrnivmodl mechanisms

In [ ]:
probe_type = "linear"
model_name = 'hay' # "hallerman"
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".")
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")

In [ ]:
_ = neuron.load_mechanisms(str(model_folder))

In [ ]:
cell = model.create(model=model_name, morph_modifier="", v_init=-75, release=True)

# probe = model.define_electrode(probe_type="linear")
probe = None

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="multiple",
    feature_file="hay_model/features_list.json",
    electrode=probe,
    protocols_with_lfp=["Step1"]
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
rheobase, rheo_protocols, rheo_responses = compute_rheobase_for_model(cell, sim=sim, step_min=0.3)

In [ ]:
ecode_protocols = generate_ecode_protocols(rheobase_current=rheobase, feature_set="soma")

In [ ]:
for p in ecode_protocols["IDthresh"].protocols:
    print(p.recordings)

In [ ]:
ecode_responses = {}
for protocol_name, protocol in ecode_protocols.items():
    print(f"Running protocol {protocol_name}")
    t_start = time.time()
    response = protocol.run(cell_model=cell, param_values={}, sim=sim)
    ecode_responses.update(response)
    print(f"Elapsed time {protocol_name}: {time.time() - t_start}")    

In [ ]:
ecode_responses

In [ ]:
plotting.plot_responses(ecode_responses, protocol_names=["PosCheops"])

In [ ]:
plt.figure()
plt.plot(responses['Step1.MEA.LFP']["time"], responses['Step1.MEA.LFP']["voltage"].T)

In [ ]:
responses['Step1.MEA.LFP']["voltage"]

In [ ]:
responses['Step1.MEA.LFP']["voltage"]

In [ ]:
ecode_responses['PosCheops-0.soma.v']["time"].values[-1]